In [2]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd
from scipy.io import arff
data = arff.loadarff('slump.arff')
df = pd.DataFrame(data[0])

covariables = df.iloc[:,0:7].values
response = df.iloc[:,7:10].values
positions = np.arange(103)

from sklearn.model_selection  import train_test_split

covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


##APPRENTISSAGE

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = numpy.empty(a.shape, dtype=a.dtype)
    shuffled_b = numpy.empty(b.shape, dtype=b.dtype)
    permutation = numpy.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

#import tensorflow as tf 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()



# Import data
# Tensorflow is finicky about shapes, so resize


X_datatrain, X_datatest, targets_train, targets_test = covariables_train, covariables_test, response_train, response_test

#les dimensions de X for training
NbrLignes_train = X_datatrain.shape[0]  ###
NbrColonnes_train= X_datatrain.shape[1] ###

#les dimensions de X for testing
NbrLignes_test = X_datatest.shape[0]  ###
NbrColonnes_test = X_datatest.shape[1] ###
#Y_data = targets 

Y_datatrain = tf.reshape(targets_train, [NbrLignes_train,3])
Y_datatest = tf.reshape(targets_test, [NbrLignes_test,3])

#build the model

# input X: batch_size x NbrColonnes, the first dimension (None) will index the data in the mini-batch
Xfill = tf.placeholder(tf.float32, shape= [None, NbrColonnes_train])
# correct answers will go here
Yfill = tf.placeholder(tf.float32, shape=[None, 3]) 
#Let's add some layers

# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 10 #200
M = 3 #160
#N = 100
#O = 30

# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([NbrColonnes_train, L], stddev=0.1))  # 
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
#W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
#B3 = tf.Variable(tf.ones([N])/10)
#W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
#B4 = tf.Variable(tf.ones([O])/10)
#W5 = tf.Variable(tf.truncated_normal([O, 6], stddev=0.1))
#B5 = tf.Variable(tf.zeros([6]))

# The model, with dropout at each layer


Y1 = tf.nn.relu(tf.matmul(Xfill, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

#Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
#Y2d = tf.nn.dropout(Y2, pkeep)

#Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
#Y3d = tf.nn.dropout(Y3, pkeep)

#Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
#Y4d = tf.nn.dropout(Y4, pkeep)

#Y_ = tf.matmul(Y4d, W5) + B5


Y_ = tf.matmul(Y1d, W2) + B2

cross_entropy = tf.losses.mean_squared_error(Yfill, Y_)

# accuracy of the trained model, between 0 (worst) and 1 (best)
#Modification de l'accuracy 

#calcul de l'accuracy, une nouvelle manière: :  

# variable learning rate
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy) 

# init
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 100.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

train_loss = []
test_loss = []

# Gradient descent loop for 500 steps
for i in range(5000):
 # Select random minibatch
    j = 1
    ##### shuffling our data::: really important
    X_datasf, Y_datasf = shuffle_in_unison(X_datatrain, Y_datatrain.eval())
    
    ##### defining a decreasing learning rate: 
    # learning rate decay
    learning_rate = 0.0001001329829992624
    #learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    ##### 
    for start, end in zip(range(0, NbrLignes_train, 52), range(20, NbrLignes_train, 52)):

             X_batch,Y_batch = X_datasf[start:end,],Y_datasf[start:end,]
             j +=1
             # Do gradient descent step
             _, loss_val_train = sess.run([train_step, cross_entropy], feed_dict={Xfill: X_batch, Yfill: Y_batch, pkeep: 0.75, lr: learning_rate})
             if i %20==0:    
                 
             
                #if j%20==0:
                 loss_val_test = sess.run(cross_entropy, feed_dict={Xfill: X_datatest, Yfill: Y_datatest.eval(), pkeep: 1.0, lr:  learning_rate})
                 train_loss.append(loss_val_train)
                 
                 test_loss.append(loss_val_test)
                 print('LOSS at step %s: - Train loss value : %f -- Test loss value : %f' % (i, loss_val_train,loss_val_test))
#print(test_loss)
#print(test_loss)
#LOSS at step 420: - Train loss value : 22.385496 -- Test loss value : 9.721767

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
LOSS at step 0: - Train loss value : 1196.576050 -- Test loss value : 1170.264771
LOSS at step 0: - Train loss value : 906.296631 -- Test loss value : 1165.759399
LOSS at step 20: - Train loss value : 875.503906 -- Test loss value : 996.524536
LOSS at step 20: - Train loss value : 867.867432 -- Test loss value : 992.395142
LOSS at step 40: - Train loss value : 1038.165405 -- Test loss value : 840.014893
LOSS at step 40: - Train loss value : 691.284912 -- Test loss value : 836.389771
LOSS at step 60: - Train loss value : 797.314026 -- Test loss value : 704.669983
LOSS at step 60: - Train loss value : 632.049927 -- Test loss value : 701.495300
LOSS at step 80: - Train loss value : 735.226685 -- Test loss value : 583.577271
LOSS at step 80: - Train loss value : 561.468567 -- Test loss value :

LOSS at step 920: - Train loss value : 425.909637 -- Test loss value : 139.982300
LOSS at step 920: - Train loss value : 298.631409 -- Test loss value : 139.749069
LOSS at step 940: - Train loss value : 229.307602 -- Test loss value : 138.202148
LOSS at step 940: - Train loss value : 217.111435 -- Test loss value : 138.448624
LOSS at step 960: - Train loss value : 262.016022 -- Test loss value : 142.595963
LOSS at step 960: - Train loss value : 208.229752 -- Test loss value : 142.487625
LOSS at step 980: - Train loss value : 282.583405 -- Test loss value : 140.946304
LOSS at step 980: - Train loss value : 216.577316 -- Test loss value : 141.231781
LOSS at step 1000: - Train loss value : 343.850037 -- Test loss value : 142.747391
LOSS at step 1000: - Train loss value : 268.309998 -- Test loss value : 142.693787
LOSS at step 1020: - Train loss value : 408.647736 -- Test loss value : 141.829910
LOSS at step 1020: - Train loss value : 267.076935 -- Test loss value : 141.780167
LOSS at step

LOSS at step 1920: - Train loss value : 238.574707 -- Test loss value : 129.841049
LOSS at step 1920: - Train loss value : 244.610214 -- Test loss value : 129.698502
LOSS at step 1940: - Train loss value : 358.130157 -- Test loss value : 131.917297
LOSS at step 1940: - Train loss value : 240.876785 -- Test loss value : 131.753845
LOSS at step 1960: - Train loss value : 346.051941 -- Test loss value : 134.675888
LOSS at step 1960: - Train loss value : 170.462830 -- Test loss value : 134.620804
LOSS at step 1980: - Train loss value : 189.264450 -- Test loss value : 128.287994
LOSS at step 1980: - Train loss value : 332.491028 -- Test loss value : 128.338486
LOSS at step 2000: - Train loss value : 290.347748 -- Test loss value : 134.178726
LOSS at step 2000: - Train loss value : 222.124481 -- Test loss value : 134.167053
LOSS at step 2020: - Train loss value : 358.808990 -- Test loss value : 134.902008
LOSS at step 2020: - Train loss value : 304.752289 -- Test loss value : 134.866943
LOSS

LOSS at step 2960: - Train loss value : 245.171738 -- Test loss value : 127.901955
LOSS at step 2960: - Train loss value : 351.296753 -- Test loss value : 127.772987
LOSS at step 2980: - Train loss value : 218.320145 -- Test loss value : 128.556656
LOSS at step 2980: - Train loss value : 202.293259 -- Test loss value : 128.367004
LOSS at step 3000: - Train loss value : 406.541870 -- Test loss value : 129.099854
LOSS at step 3000: - Train loss value : 265.323669 -- Test loss value : 129.427155
LOSS at step 3020: - Train loss value : 163.468918 -- Test loss value : 130.874664
LOSS at step 3020: - Train loss value : 260.467834 -- Test loss value : 130.851608
LOSS at step 3040: - Train loss value : 285.740936 -- Test loss value : 128.036133
LOSS at step 3040: - Train loss value : 162.991318 -- Test loss value : 128.130188
LOSS at step 3060: - Train loss value : 449.164062 -- Test loss value : 125.553284
LOSS at step 3060: - Train loss value : 157.160553 -- Test loss value : 125.516342
LOSS

LOSS at step 3980: - Train loss value : 184.743546 -- Test loss value : 125.535110
LOSS at step 3980: - Train loss value : 315.678345 -- Test loss value : 125.573402
LOSS at step 4000: - Train loss value : 279.919739 -- Test loss value : 127.079697
LOSS at step 4000: - Train loss value : 177.311005 -- Test loss value : 127.493446
LOSS at step 4020: - Train loss value : 234.219208 -- Test loss value : 133.101929
LOSS at step 4020: - Train loss value : 256.451477 -- Test loss value : 132.952881
LOSS at step 4040: - Train loss value : 232.720245 -- Test loss value : 131.789963
LOSS at step 4040: - Train loss value : 317.015289 -- Test loss value : 131.896683
LOSS at step 4060: - Train loss value : 241.096481 -- Test loss value : 127.905876
LOSS at step 4060: - Train loss value : 260.745331 -- Test loss value : 128.203690
LOSS at step 4080: - Train loss value : 209.573669 -- Test loss value : 123.813675
LOSS at step 4080: - Train loss value : 215.845688 -- Test loss value : 123.734749
LOSS

# Let us compute the aRRMSE

In [3]:
learning_rate = 0.0001001329829992624

Y_testpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatest, tf.float32).eval(), Yfill: tf.cast(Y_datatest, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})
#numpy.savetxt('Y_testpred_with_stock_%s_and_ai_%s_modelcut_%s_vartimeframe_%s.txt'%(stock, ai, cut, vartimeframe), Y_testpred, fmt='%f')
Y_trainpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatrain, tf.float32).eval(), Yfill: tf.cast(Y_datatrain, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})


In [4]:
Y_testpred

array([[19.910864, 53.838516, 40.455544],
       [18.839981, 52.02758 , 38.427677],
       [15.699167, 42.041897, 31.618938],
       [16.038692, 42.742336, 32.426117],
       [14.491761, 39.834255, 29.272587],
       [14.654827, 40.90637 , 29.597027],
       [13.763636, 36.87045 , 27.866917],
       [15.696049, 42.071995, 31.588068],
       [16.743916, 47.289356, 33.97391 ],
       [16.108744, 43.038776, 32.495644],
       [19.139694, 51.445244, 38.959476],
       [21.174046, 57.59976 , 42.879658],
       [21.17463 , 58.040955, 42.919914],
       [14.756723, 39.54933 , 29.950932],
       [16.716276, 45.23922 , 34.046696],
       [13.846725, 37.386703, 27.868904],
       [16.96393 , 46.222107, 34.53545 ],
       [18.240725, 50.326744, 36.972187],
       [16.324738, 46.180153, 33.06686 ],
       [17.366587, 47.06668 , 35.038673],
       [16.107454, 42.79973 , 32.65476 ]], dtype=float32)

In [5]:
from numpy import linalg as LA

In [6]:

aRRMSE_list = []

for i in [0,1,2] :
    
    Errors_test =  Y_testpred[:,i] - response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    ytrain_mean_i = np.tile(Y_trainpred[:,i].mean(axis = 0), (covariables_test.shape[0], 1))

    Errors_relative = ytrain_mean_i - response_test[:,i]

    Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%Final_SPNNR_aRRMSE)

    aRRMSE_list.append(Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 20.147923
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 19.145161
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 17.259488


In [7]:
np.mean(aRRMSE_list)

18.850857264121945

# Check if old way calculation is wrong

In [22]:
Errors_test =  Y_testpred - response_test


print('<*********************************Let us check the aRRMSE*********************************>')

ytrain_mean = np.tile(Y_trainpred.mean(axis = 0), (response_test.shape[0], 1))

Errors_relative = ytrain_mean - response_test

Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

print("Final_Partition aRRMSE is : %f"%Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final_Partition aRRMSE is : 84.178972


*** The answer is yes ***

# Checking True aRRMSE of S-SPNNR on Slump

In [28]:
spnnr_covariables_train, spnnr_covariables_test, spnnr_response_train, spnnr_response_test,spnnr_positions_train,spnnr_positions_test = train_test_split(covariables, response,positions, test_size=0.33, random_state=42)

In [32]:
spnnr_Y_testpred =np.loadtxt('All_Y_test_predictions.out', delimiter=',' )
spnnr_Y_trainpred =np.loadtxt('All_Y_train_predictions.out', delimiter=',' )

In [33]:
spnnr_aRRMSE_list = []

for i in [0,1,2] :
    
    spnnr_Errors_test =  spnnr_Y_testpred[:,i] - spnnr_response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    spnnr_ytrain_mean_i = np.tile(spnnr_Y_trainpred[:,i].mean(axis = 0), (spnnr_covariables_test.shape[0], 1))

    spnnr_Errors_relative = spnnr_ytrain_mean_i - spnnr_response_test[:,i]

    spnnr_Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(spnnr_Errors_test)**2)/(LA.norm(spnnr_Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%spnnr_Final_SPNNR_aRRMSE)

    spnnr_aRRMSE_list.append(spnnr_Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 27.482091
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 21.644258
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 16.574829


In [34]:
np.mean(spnnr_aRRMSE_list)

21.90039258118671